# Model comparison
This notebook evaluates a model according to an agreed upon metric in order to compare results with the GP models.

In [1]:
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt

In [2]:
"""
Some loss functions
"""
def basic_mae(preds, labels):
    acc = 0 
    n = len(preds)
    for i in range(n):
        e = abs(preds[i] - labels.values[i])
        acc += e
        #print("Errs:", e1, "\t", e2, "\tDiff:", e1-e2, "\t - accs:", acc1, acc2)

    acc /= n
    return acc

def mape(preds, labels):
    acc = 0
    n = len(preds)
    for i in range(n):
        acc+=abs((labels.values[i] - preds[i])/labels.values[i])
    
    res = acc/n
    return res

Pick the name for your model

In [6]:
model_name = 'ANN_M4'

In [7]:
data = pds.read_pickle(model_name + '.pkl')
data.head()

,seg,journey,speed,pred,label
0,1,15,0.0,116.291199,191.0
1,1,15,0.0,116.291817,190.0
2,1,15,0.0,116.292236,189.0
3,1,15,0.0,116.292519,188.0
4,1,15,0.0,116.292557,187.0


In [9]:
segments = data.seg.unique()
journeys = data.journey.unique()

In [28]:
mae_matrix = pds.DataFrame()
mape_matrix = pds.DataFrame()
#Intervals to be evaluated
intervals = [20,40,60,80]
for s in segments:
    mae_accum = [0] * len(intervals)
    mape_accum = [0] * len(intervals)
    for j in journeys:
        #Extract segment s from journey j
        seg_data = data[(data['seg'] == s) & (data['journey'] == j)]
        seg_data = seg_data.reset_index(drop=True)
        #For each interval, accumulate the mae/mape
        for i, val in enumerate(intervals):
            n = round(len(seg_data)*(val/100))
            mae_accum[i] += basic_mae(seg_data['pred'][:n], seg_data['label'][:n])
            mape_accum[i] += mape(seg_data['pred'][:n], seg_data['label'][:n])
    #Calculate the average mae/mape
    for i, val in enumerate(intervals):
        #TODO: Store values in dataframes and present in a nicer way
        print('seg:', s,'interval:', val,': Average MAE: ', mae_accum[i]/len(journeys))
        print('seg:', s,'interval:', val,': Average MAPE: ', mape_accum[i]/len(journeys))
    

seg: 1 interval: 20 : Average MAE:  41.88530691873374
seg: 1 interval: 20 : Average MAPE:  0.2764002048442867
seg: 1 interval: 40 : Average MAE:  33.308148586344515
seg: 1 interval: 40 : Average MAPE:  0.23104016854660256
seg: 1 interval: 60 : Average MAE:  29.81192264430808
seg: 1 interval: 60 : Average MAPE:  0.2405465446983472
seg: 1 interval: 80 : Average MAE:  28.48169162554499
seg: 1 interval: 80 : Average MAPE:  0.277461182725158
seg: 2 interval: 20 : Average MAE:  6.453162875024431
seg: 2 interval: 20 : Average MAPE:  0.13508645049064172
seg: 2 interval: 40 : Average MAE:  5.52148148594927
seg: 2 interval: 40 : Average MAPE:  0.13612380814360256
seg: 2 interval: 60 : Average MAE:  4.499619013003597
seg: 2 interval: 60 : Average MAPE:  0.13013608198822474
seg: 2 interval: 80 : Average MAE:  3.7213204307376846
seg: 2 interval: 80 : Average MAPE:  0.1361255178582258
seg: 3 interval: 20 : Average MAE:  6.177085735496756
seg: 3 interval: 20 : Average MAPE:  0.10019259975715149
seg: 